In [1]:
# standard libraries
import numpy as np
import math
from pprint import pprint as pp
from matplotlib.pyplot import *
%matplotlib inline

In [2]:
# user library
from bayesee.detector.spotter import *
from bayesee.imaging.filter import *
from bayesee.imaging.image import *
from bayesee.operation.nb2d import *
from bayesee.operation.dp import *
from bayesee.operation.mathfunc import *

In [3]:
# stimulus parameters
row = col = 100
i_row = i_col = 128
ppd = 120
cpd = 4
cpi_v = row * cpd / ppd
field_ratio = 8

In [4]:
# simulation parameters
n_bin_spat_sim = 5
n_bin_amp_sim = 3
n_bin_background = 1000
n_background = n_bin_background * n_bin_amp_sim * n_bin_spat_sim
background_mean = 0
background_std = 0.204

In [5]:
# templates
cosine_template = hann_window(row, col, row/2) * cosine_wave(row, col, (cpi_v,0))
cosine_template /= nb2dot(cosine_template, cosine_template)

In [6]:
# backgrounds
one_f_0_noise,_ = power_noise_patches(i_row, i_col, i_row*field_ratio, i_col*field_ratio, 0, n_background, background_mean, background_std)
one_f_noise,_ = power_noise_patches(i_row, i_col, i_row*field_ratio, i_col*field_ratio, -1, n_background, background_mean, background_std)
one_f_3_noise,_ = power_noise_patches(i_row, i_col, i_row*field_ratio, i_col*field_ratio, -3, n_background, background_mean, background_std)

backgrounds = [one_f_0_noise, one_f_noise, one_f_3_noise]

In [ ]:
# spotters
csf = {'ppd': ppd}
uncer_std = 0.083*ppd/8
uncer={'size': uncer_std * 3, 'func': lambda x: np.exp(-(x/uncer_std)**2/2) / np.sqrt(2*np.pi*uncer_std), 'info': 'prior', 'focus': 'max'}

TM = Spotter(method='TM')
WTM1 = Spotter(method='WTM', whiten=1)
WTM3 = Spotter(method='WTM', whiten=3)
WTM = [TM, WTM1, WTM3]
UTM = UncertainSpotter(method='UTM', uncer=uncer)
UWTM1 = UncertainSpotter(method='UWTM', whiten=1, uncer=uncer)
UWTM3 = UncertainSpotter(method='UWTM', whiten=3, uncer=uncer)
UWTM = [UTM, UWTM1, UWTM3]
models = [WTM, UWTM]

In [ ]:
# window
hann = flat_top_hann_window(row, col, [0.45 * row, 0.5 * row])

In [ ]:
# simulation function
def power_noise_spat_sim_amp_sim(template, background, window, n_amps):
    n_background = background.shape[2]
    spat_sim = np.zeros((n_background*2*n_amps,))
    amp_sim = np.zeros((n_background*2*n_amps,))
    
    for i in range(n_background):
        spat_sim[2*n_amps*i:2*n_amps*(i+1)] = spatial_cosine_similarity(cut_center(background[:,:,i],template), template, window)
        amp_sim[2*n_amps*i:2*n_amps*(i+1)] = amplitude_cosine_similarity(cut_center(background[:,:,i],template), template, window)
        
    return spat_sim, amp_sim

def detection_model_power_noise(model, template, background, amp_init, n_amps):
    amp = np.ones((background.shape[2],)) * amp_init
    stimulus = np.zeros_like(amp)
    response = np.zeros_like(amp)
    
    j = 0
    for i in range(background.shape[2]):
        if j == n_amps:
            j = 0
            
        amp[i] *= 2**j
        
        if i%2:
            stimulus[i] = 0
            response[i] = model.give_response(background[:,:,i], template)
        else:
            stimulus[i] = 1
            response[i] = model.give_response(add_to_center(background[:,:,i], amp[i]*template), template)
            
        j += 1
        
    return amp, stimulus, response

In [ ]:
# calculate spat sims and amp sims
n_amps = 5
amp_all = np.zeros((len(models), len(backgrounds), n_background * 2 * n_amps))
stimulus_all = np.zeros_like(amp_all)
response_all = np.zeros_like(amp_all)
spat_sim_all = np.zeros_like(amp_all)
amp_sim_all = np.zeros_like(amp_all)

for b, background in enumerate(backgrounds):
    spat_sim_all[0,b,:], amp_sim_all[0,b,:] = power_noise_spat_sim_amp_sim(cosine_template, background, hann, n_amps)
    for m, model in enumerate(models):
        if m > 0:
            spat_sim_all[m,b,:], amp_sim_all[m,b,:] = spat_sim_all[0,b,:], amp_sim_all[0,b,:]

In [ ]:
# calculate thresholds
amp_inits = [[0.2, 0.5, 0.3], [0.2, 0.75, 0.3], [0.2, 1, 0.3]]

th_all = np.zeros((len(models), len(backgrounds), n_bin_spat_sim, n_bin_amp_sim))

def equal_frequency_bins_2d(x, y, bins):
    # x is binned first, y is then binned in the bins of x
    bins_x = np.interp(np.linspace(0, len(x), bins[0] + 1), np.arange(len(x)), np.sort(x))
    bins_y = np.zeros((bins[0], bins[1]+1))
    for i in range(bins[0]):
        b_y = y[(x>=bins_x[i]) & (x<bins_x[i+1])]
        bins_y[i,:] = np.interp(np.linspace(0, len(b_y), bins[1] + 1), np.arange(len(b_y)), np.sort(b_y))
        
    return bins_x, bins_y

bin_edges_amp_sim_all = np.zeros((len(models), len(backgrounds), n_bin_amp_sim+1))
bin_edges_spat_sim_all = np.zeros((len(models), len(backgrounds), n_bin_amp_sim, n_bin_spat_sim+1))

for b,background in enumerate(backgrounds):
    repeated_background = np.repeat(background, 2*n_amps, axis=2)
    b_e_amp_sim, b_e_spat_sim = equal_frequency_bins_2d(amp_sim_all[0,b,:], spat_sim_all[0,b,:], [n_bin_amp_sim, n_bin_spat_sim])
    for m,model in enumerate(models):
        bin_edges_amp_sim_all[m,b,:], bin_edges_spat_sim_all[m,b,:,:] = b_e_amp_sim, b_e_spat_sim
        for i in range(n_bin_amp_sim):
            for j in range(n_bin_spat_sim):
                bin_idx = (amp_sim_all[m,b,:] >= b_e_amp_sim[i]) & (amp_sim_all[m,b,:] < b_e_amp_sim[i+1]) & (spat_sim_all[m,b,:] >= b_e_spat_sim[i,j]) & (spat_sim_all[m,b,:] < b_e_spat_sim[i,j+1])
                amp_all[m,b,bin_idx], stimulus_all[m,b,bin_idx], response_all[m,b,bin_idx] = bin_amp, bin_stimulus, bin_response = detection_model_power_noise(model[b], cosine_template, repeated_background[:,:,bin_idx], amp_inits[m][b], n_amps)
                if m < 2:
                    th_all[m,b,j,i] = linear_cont_th(bin_amp, bin_stimulus, bin_response)
                else:
                    th_all[m,b,j,i] = uncertain_cont_th(bin_amp, bin_stimulus, bin_response)
                    
np.savez('cosine-spat-sim-amp-sim-1-f-noise-field-ideal', amp_all=amp_all, stimulus_all=stimulus_all, response_all=response_all, spat_sim_all=spat_sim_all, amp_sim_all=amp_sim_all, bin_edges_spat_sim_all=bin_edges_spat_sim_all, bin_edges_amp_sim_all=bin_edges_amp_sim_all, th_all = th_all)

In [ ]:
# plot function
def plot_th_bin2d(bin_edges_spat_sim_all, bin_edges_amp_sim_all, th_all, pargs):
    db_th_all = decibel(th_all)
    
    n_models, n_backgrounds, n_bin_amp_sim,_  = bin_edges_spat_sim_all.shape
    n_rows, n_cols = pargs['n_rows'], pargs['n_cols']
    fig, axs = subplots(nrows=n_rows, ncols=n_cols, figsize=pargs['figsize'], constrained_layout=True)
    
    for m in range(n_models):
        for b in range(n_backgrounds):
            x_max = np.abs(bin_edges_spat_sim_all[:,b,:,:]).max()
            
            bin_centers_amp_sim = (bin_edges_amp_sim_all[m,b,:-1]+bin_edges_amp_sim_all[m,b,1:])/2
            bin_centers_spat_sim = (bin_edges_spat_sim_all[m,b,:,:-1]+bin_edges_spat_sim_all[m,b,:,1:])/2
            
            if n_models == 1 or n_backgrounds == 1:
                idxes = b if n_models == 1 else m
            else:
                idxes = m, b
            
            for i in range(n_bin_amp_sim):
                axs[idxes].scatter(bin_centers_spat_sim[i,:], db_th_all[m,b,:,i], s=pargs['markersize'], marker=pargs['markers'][i], edgecolor=pargs['colors'][i], facecolor=pargs['colors'][i], linewidths=pargs['linewidth'], label=f'acs:{bin_centers_amp_sim[i]:.2f}')
                axs[idxes].plot(bin_centers_spat_sim[i,:], db_th_all[m,b,:,i], color=pargs['colors'][i], linewidth=pargs['linewidth'])
                axs[idxes].set_xlim(-x_max, x_max)
                
            axs[idxes].text(0.1,0.1,pargs['legends'][m][b], transform=axs[idxes].transAxes, c='k', fontsize=pargs['fontsizes'][2])
            axs[idxes].legend(loc='upper right', fontsize=pargs['fontsizes'][2])
        
            axs[idxes].tick_params(axis='x', which='both', direction='out', length=0, width=0,pad=5, labelsize=pargs['fontsizes'][2], labelbottom=True, labeltop=False, grid_color='k', grid_alpha=1, grid_linewidth=1, grid_linestyle='--')
            axs[idxes].grid(visible=True, which='minor', axis='x', linestyle='--', linewidth=pargs['linewidth'])
            axs[idxes].tick_params(axis='y', which='major', direction='out', length=12, width=4, pad=3, labelsize=pargs['fontsizes'][2], left=True, right=True, labelleft=True, labelright=True)
            axs[idxes].tick_params(axis='y', which='minor', direction='out', length=8, width=4, left=True, right=True, labelleft=False, labelright=False)

    fig.text(0.5, -0.075, pargs['x_label'], ha='center', fontsize=pargs['fontsizes'][0])
    fig.text(-0.05, 0.5, pargs['y_label'], va='center', rotation='vertical', fontsize=pargs['fontsizes'][0])
    savefig('th_spat_sim_amp_sim_' + pargs['plot_name'] + '_ideal.svg', dpi=300, bbox_inches='tight')
    close()


In [ ]:
# plot
data = np.load('cosine-spat-sim-amp-sim-1-f-noise-field-ideal.npz')
amp_all, stimulus_all, response_all, spat_sim_all, amp_sim_all, bin_edges_spat_sim_all, bin_edges_amp_sim_all, th_all = data['amp_all'], data['stimulus_all'], data['response_all'], data['spat_sim_all'], data['amp_sim_all'], data['bin_edges_spat_sim_all'], data['bin_edges_amp_sim_all'], data['th_all']

n_row = len(models)
n_col = len(backgrounds)
pargs = {'plot_name': 'cosine', 'figsize': (20, 8), 'n_rows': n_row, 'n_cols': n_col, 'fontsizes': [36, 15, 12], 'x_label': 'Spatial Cosine Similarity', 'y_label': 'Amplitude Cosine Similarity', 'legends': [['WTM | white', 'WTM | 1/f', 'WTM | 1/f^3'], ['UWTM | white', 'UWTM | 1/f', 'UWTM | 1/f^3']], 'markers': ['o', 'o', 'o', 'o', 'o', 'o', 'o'], 'colors':['r', 'g', 'b', 'c', 'm', 'y', 'k'], 'markersize': 100, 'linewidth': 2}
plot_th_bin2d(bin_edges_spat_sim_all, bin_edges_amp_sim_all, th_all, pargs)